In [1]:
import logging
import os
from annoy import AnnoyIndex
import random
import pandas as pd
import random

from preprocessor import Preprocessor
from user_model import UserModel
from recommender import Recommender

data_source_dir = '../data' 
output_name = '0504'
news_train_dates = {'start':24,'end':26}
news_candidates_dates = {'start':25,'end':26}
news_train_paths = [os.path.join(data_source_dir,'event-201904{}.pkl'.format(str(x).zfill(2))) for x in range(news_train_dates['start'], news_train_dates['end'])]
news_candidates_paths = [os.path.join(data_source_dir,'event-201904{}.pkl'.format(str(x).zfill(2))) for x in range(news_candidates_dates['start'], news_candidates_dates['end'])]
news_vec_pool_pah = os.path.join(data_source_dir,'news_vec_dC_off.pkl')

def logging_setup():
    logging.basicConfig(
        format='%(asctime)s | %(levelname)s | %(name)s: %(message)s',
        level=logging.INFO, datefmt='%m-%d %H:%M:%S'
    )

Using TensorFlow backend.
/Users/Yu/anaconda3/envs/recommender/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
logging_setup()
logging.info('=============== START RUNNING ===============')

05-06 14:56:28 | INFO | root: =============== START RUNNING ===============


In [3]:
preprocessor = Preprocessor(dir=data_source_dir,name=output_name)
preprocessor.load_news_vec_pool(news_vec_pool_pah)
preprocessor.load_datas_for_user_model(news_train_paths)

05-05 15:50:00 | INFO | preprocessor: loading 2 days events for model training...


In [ ]:
preprocessor.build_user_to_news_history_custom()
preprocessor.build_sampling_pool(top = 5000, at_least = 10)
preprocessor.build_vec_pairs_from_history(at_least=10)
preprocessor.load_datas_for_candidates_pool(news_candidates_paths)
preprocessor.build_candidates_pool(top = 5000, at_least = 10)

05-05 15:50:16 | INFO | preprocessor: building user_to_news_list...
05-05 15:50:16 | INFO | preprocessor: - cleanning data...
05-05 15:50:22 | INFO | preprocessor: - drop duplicating user/item pairs...
05-05 15:50:40 | INFO | preprocessor: - sorting events...
05-05 15:50:50 | INFO | preprocessor: - grouping news_id by user_id...
05-05 15:50:50 | INFO | preprocessor: - convert news history into dictionary...
05-05 15:51:29 | INFO | preprocessor: - saving user_to_news_list...
05-05 15:51:37 | INFO | preprocessor: - complete saving user_to_news_list.
05-05 15:51:38 | INFO | preprocessor: building sampling_pool...
05-05 15:51:38 | INFO | preprocessor: - cleanning data...
05-05 15:51:40 | INFO | preprocessor: - drop duplicating user/item pairs...
05-05 15:51:57 | INFO | preprocessor: - complete building sampling_pool.
05-05 15:51:58 | INFO | preprocessor: building user_news_vec_pairs...
05-05 15:54:02 | INFO | preprocessor: 260966 user_news_vec_pairs builded...
05-05 16:01:23 | INFO | prepr

In [ ]:
preprocessor.build_user_to_news_history()

In [ ]:
k,v = zip(*preprocessor.user_to_news_pos_vec.items())
len(preprocessor.user_to_news_pos_vec)

In [3]:
user_model = UserModel(dir=data_source_dir,name=output_name)
user_model.load_news_history()
user_model.model_training(start=0,
                          items=10,
                          N=10,
                          epochs=20, 
                          batch_size=32, 
                          model_type='GRU', 
                          validation_split=0.1,
                          init_rnn_by_avg=False,
                         neg_sampling=False)
user_model.save_user_vec_pool()
user_model.save_user_model()
user_model.load_user_model()
user_model.load_news_history()

05-06 15:37:00 | INFO | user_model: loading reading history...
05-06 15:37:46 | INFO | user_model: - users: 260966
05-06 15:37:46 | INFO | user_model: - complete loading reading history...
05-06 15:37:46 | INFO | user_model: building user-vectors...
05-06 15:37:46 | INFO | user_model: - news-threshold: 10
05-06 15:37:52 | INFO | user_model: - qualified users: 259155
05-06 15:37:53 | INFO | user_model: - complete building news vectors list
05-06 15:40:08 | INFO | user_model: ******** model setting *********
05-06 15:40:08 | INFO | user_model: *****
05-06 15:40:09 | INFO | user_model: *****   model:GRU
05-06 15:40:09 | INFO | user_model: *****   init :False
05-06 15:40:09 | INFO | user_model: *****   negs :False
05-06 15:40:09 | INFO | user_model: *****
05-06 15:40:09 | INFO | user_model: ********************************


Train on 233239 samples, validate on 25916 samples
Epoch 1/20
227552/233239 [============================>.] - ETA: 41s - loss: 0.0311

05-06 16:08:07 | ERROR | root: Internal Python error in the inspect module.
Below is the traceback from this internal error.

05-06 16:08:07 | INFO | root: 
Unfortunately, your original traceback can not be constructed.


KeyboardInterrupt



Traceback (most recent call last):
  File "/Users/Yu/anaconda3/envs/recommender/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-d6765c388804>", line 11, in <module>
    neg_sampling=False)
  File "/Users/Yu/Documents/Course/NTU_MSlab/MSlab_news_rec/embedding-based-recommender/src/user_model.py", line 164, in model_training
    model.fit(X_train,Y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split, callbacks=[callback])
  File "/Users/Yu/anaconda3/envs/recommender/lib/python3.6/site-packages/keras/engine/training.py", line 1657, in fit
    validation_steps=validation_steps)
  File "/Users/Yu/anaconda3/envs/recommender/lib/python3.6/site-packages/keras/engine/training.py", line 1204, in _fit_loop
    ins_batch = _slice_arrays(ins, batch_ids)
  File "/Users/Yu/anaconda3/envs/recommender/lib/python3.6/site-packages/keras/engine/training.py", line 

In [ ]:
recommender = Recommender(dir=data_source_dir,name=output_name)
recommender.load_vec_pool()
recommender.build_annoy_indexer()

In [ ]:
user_id, user_vec = random.choice(list(recommender.user_vec_pool.items()))
news_id, news_vec = random.choice(list(recommender.news_vec_pool.items()))
rec_list = recommender.get_ranking_list_by_both(user_id,news_id, realtime=True, items=20)

print(rec_list)
# r_list_u = recommender.get_ranking_list_by_annoy(user_id=u_id,items=10)
# r_list_n = recommender.get_ranking_list_by_annoy(news_id=n_id,items=10)

In [ ]:
#### EXPERIMENT

In [ ]:
a = pd.read_pickle(os.path.join(data_source_dir,'event-20190424.pkl'))

In [ ]:
len(a)

In [ ]:
print('start')
dic = {}
j=0
for i in a_list:
    continue
print('done')

In [ ]:
a.head(3)

In [ ]:
print('start')
for i in range(7000000):
    continue
print('done')

In [ ]:
a.head()

In [ ]:
event_time = 'event_timestamp'
news_id    = 'page.item.id'
user_id    = 'eds_id'
n_list = a[news_id].tolist()
u_list = a[user_id].tolist()
t_list = a[event_time].tolist()

In [ ]:
logging.info('=============== START RUNNING ===============')


In [ ]:
logging.info('=============== Start Sorting ===============')
t_list,u_list,n_list = zip(*sorted(zip(t_list,u_list,n_list), reverse=True))
logging.info('=============== Done Sorting ===============')

In [ ]:
t_list[0]

In [ ]:
t_list[-1]